In [65]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pip install git+https://github.com/openai/CLIP.git

In [ ]:
pip install clip

In [68]:
pip install emoji

In [69]:
pip install grapheme

In [ ]:
pip install rouge-score

In [71]:
import matplotlib.pyplot as plt
import torch
import clip
from PIL import Image
from tqdm import tqdm

In [72]:
import pandas as pd
import numpy as np
import ast
import grapheme
import emoji
import torch
from sklearn.metrics.pairwise import cosine_similarity

Basic CLIP for single Emoji

In [ ]:
datapath = "/content/drive/MyDrive/CLIP/ELCo.csv"
df = pd.read_csv(datapath)

df["Description"] = df["Description"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

text_data = [" ".join(desc[:5]) for desc in df["Description"]]

df["EM_Text"] = df["EM"].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
emoji_char_data = df["EM"].tolist()  

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

text_features = model.encode_text(
    clip.tokenize(text_data, truncate=True).to(device)
).detach().cpu().numpy()

def retrieve_top_k_emojis(query_text, k=3):
    query_embedding = model.encode_text(clip.tokenize([query_text], truncate=True).to(device)).detach().cpu().numpy()
    similarities = cosine_similarity(query_embedding, text_features)[0]
    top_k_indices = np.argsort(-similarities)[:k]
    top_k_emojis = []
    for i in top_k_indices:
        emojis = list(grapheme.graphemes(emoji_char_data[i])) 
        top_k_emojis.append(emojis[0])  
    return top_k_emojis

# Example 
query = "speaking_head"
predicted_emojis = retrieve_top_k_emojis(query, k=3)
print(f"Input: {query} → Predicted Emojis: {predicted_emojis}")

Input: speaking_head → Predicted Emojis: ['🗣️', '🗣️', '🗣️']


In [ ]:
# encoder evaluation using cosine similarity
def evaluate_encoder():
    similarities = []

    for text, emoji_gt in zip(df["Description"], df["EM_Text"]):
        predicted_emoji_text = retrieve_top_k_emojis(" ".join(text[:5]), k=1)[0] 

        if predicted_emoji_text and emoji_gt:
            predicted_embedding = model.encode_text(clip.tokenize([predicted_emoji_text]).to(device)).detach().cpu().numpy()
            ground_truth_embedding = model.encode_text(clip.tokenize([emoji_gt]).to(device)).detach().cpu().numpy()
            sim_score = cosine_similarity(predicted_embedding, ground_truth_embedding)[0][0]
            similarities.append(sim_score)

    if similarities:
        avg_similarity = np.mean(similarities)
        print(f"Encoder Cosine Similarity: {avg_similarity:.4f}")
    else:
        print("No valid similarity scores computed!")

evaluate_encoder()

✅ncoder Cosine Similarity: 0.7457


In [ ]:

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer

smoothie = SmoothingFunction().method4

def emoji_to_text(emoji_char):
    return emoji.demojize(emoji_char, delimiters=(" ", " "))

def evaluate_decoder():
    bleu_scores, meteor_scores, rouge_scores = [], [], []

    for text, emoji_gt in zip(df["Description"], df["EM"]):
        predicted_emoji = retrieve_top_k_emojis(" ".join(text[:5]), k=1)[0]

        predicted_text = emoji_to_text(predicted_emoji)
        ground_truth_text = emoji_to_text(emoji_gt)

        predicted_tokens = predicted_text.split()
        ground_truth_tokens = ground_truth_text.split()

        bleu = sentence_bleu([ground_truth_tokens], predicted_tokens, smoothing_function=smoothie)

        meteor = meteor_score([ground_truth_tokens], predicted_tokens)

        rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        rouge_l = rouge.score(" ".join(ground_truth_tokens), " ".join(predicted_tokens))['rougeL'].fmeasure

        bleu_scores.append(bleu)
        meteor_scores.append(meteor)
        rouge_scores.append(rouge_l)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_meteor = sum(meteor_scores) / len(meteor_scores)
    avg_rouge = sum(rouge_scores) / len(rouge_scores)

    print(f"BLEU Score: {avg_bleu:.4f}")
    print(f"METEOR Score: {avg_meteor:.4f}")
    print(f"ROUGE-L Score: {avg_rouge:.4f}")


evaluate_decoder()

BLEU Score: 0.3126
METEOR Score: 0.2400
ROUGE-L Score: 0.6030


Emoji comibination using CLIP

In [110]:
datapath = "/content/drive/MyDrive/CLIP/ELCo_adapted_edited.csv"
df = pd.read_csv(datapath)
df

,text,emoji
0,"""Dressing for success in big business leads to...",👔📈
1,The big business is making a lot of money!,🏢🤑🤑
2,A businessman collaborates with a tech expert ...,👨‍💻🤝
3,The office building is bustling with teamwork ...,🏢🧑‍🤝‍🧑🧑‍🤝‍🧑🧑‍🤝‍🧑
4,The savvy businesswoman is making a fortune in...,👩‍💻🤑
...,...,...
1650,"""Listen closely to the wise words being shared.""",🔊🗣️
1651,"""Here’s some sound advice that you can really ...",👍🏻🎙🗣
1652,"A boy is giving thoughtful advice to a girl, a...",👦🗣️👩🤔👍
1653,"""That's great advice, and I'm all ears!""",👍🧏‍♀️


In [ ]:

df["Emoji_Text"] = df["emoji"].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

text_data = df["text"].tolist() 
emoji_text_data = df["Emoji_Text"].tolist() 
emoji_char_data = df["emoji"].tolist() 

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

text_features = model.encode_text(
    clip.tokenize(text_data, truncate=True).to(device)
).detach().cpu().numpy()


def retrieve_top_k_emoji_combinations(query_text, k=3, threshold=0.2):
    query_embedding = model.encode_text(clip.tokenize([query_text], truncate=True).to(device)).detach().cpu().numpy()
    similarities = cosine_similarity(query_embedding, text_features)[0]

    top_k_indices = np.argsort(-similarities)[:k * 2]

    top_k_combos = []
    for i in top_k_indices:
        if similarities[i] < threshold:
            continue
        emoji_combo = emoji_char_data[i] 
        if emoji_combo not in top_k_combos:
            top_k_combos.append(emoji_combo)
        if len(top_k_combos) >= k:
            break

    return top_k_combos  


# Example 
query = "The big business is making a lot of money!"
predicted_emojis = retrieve_top_k_emoji_combinations(query, k=3)
print(f"Input: {query} → Predicted Emojis: {predicted_emojis}")

#  evaluate encoder using cosine similarity
def evaluate_encoder():
    similarities = []

    for text, emoji_gt in zip(df["text"], df["Emoji_Text"]):
        predicted_emoji_text = " ".join(retrieve_top_k_emojis(text, k=1))

        if predicted_emoji_text and emoji_gt:
            predicted_embedding = model.encode_text(clip.tokenize([predicted_emoji_text]).to(device)).detach().cpu().numpy()
            ground_truth_embedding = model.encode_text(clip.tokenize([emoji_gt]).to(device)).detach().cpu().numpy()

            sim_score = cosine_similarity(predicted_embedding, ground_truth_embedding)[0][0]
            similarities.append(sim_score)

    if similarities:
        avg_similarity = np.mean(similarities)
        print(f"Encoder Cosine Similarity: {avg_similarity:.4f}")
    else:
        print("No valid similarity scores computed!")


evaluate_encoder()



Input: The big business is making a lot of money! → Predicted Emojis: ['🏢🤑🤑', '🤏💲', '🤏💷']
Encoder Cosine Similarity: 0.7457


In [76]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def evaluate_decoder():
    bleu_scores, meteor_scores, rouge_scores = [], [], []
    smoothing = SmoothingFunction().method1
    rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)

    for text, emoji_gt in zip(df["text"], df["emoji"]):
        predicted_emoji = retrieve_top_k_emoji_combinations(text, k=1)[0]  

        if predicted_emoji and emoji_gt:
            predicted_tokens = list(grapheme.graphemes(predicted_emoji))
            reference_tokens = list(grapheme.graphemes(emoji_gt))

            # BLEU
            bleu = sentence_bleu([reference_tokens], predicted_tokens, smoothing_function=smoothing)
            bleu_scores.append(bleu)

            # METEOR
            meteor = meteor_score([reference_tokens], predicted_tokens)
            meteor_scores.append(meteor)

            # ROUGE-L using demojized emoji text
            predicted_text = emoji.demojize(predicted_emoji, delimiters=(" ", " "))
            ground_truth_text = emoji.demojize(emoji_gt, delimiters=(" ", " "))
            rouge_l = rouge.score(ground_truth_text, predicted_text)['rougeL'].fmeasure
            rouge_scores.append(rouge_l)

    print(f"\n Decoder Evaluation Results:")
    print(f"BLEU Score: {np.mean(bleu_scores):.4f}")
    print(f"METEOR Score: {np.mean(meteor_scores):.4f}")
    print(f"ROUGE-L Score: {np.mean(rouge_scores):.4f}")
    
evaluate_decoder()


 Decoder Evaluation Results:
BLEU Score: 0.4572
METEOR Score: 0.9157
ROUGE-L Score: 0.9934


In [ ]:
def evaluate_hamming_loss():
    losses = []

    for text, gt in zip(df["text"], df["emoji"]):
        predicted = retrieve_top_k_emoji_combinations(text, k=1)[0] 
        pred_set = set(grapheme.graphemes(predicted))
        gt_set = set(grapheme.graphemes(gt))

        union_len = len(pred_set.union(gt_set))
        xor_len = len(pred_set.symmetric_difference(gt_set))

        if union_len > 0:
            losses.append(xor_len / union_len)
        else:
            losses.append(0)  

    hamming = np.mean(losses)
    print(f"Hamming Loss: {hamming:.4f}")

evaluate_hamming_loss()

Hamming Loss: 0.0093
